# [프로젝트2] ARIMA 모델로 이상치 탐지하기

---

## 프로젝트 목표
---

1. MME 데이터가 정상성을 만족하는지 확인 
2. ARIMA 모델의 최적의 파라미터를 찾기
3. ARIMA 모델을 이용하여 MME 장비의 이상치 탐지

## 프로젝트 목차

--- 

<div class="toc"><ul class="toc-item"><li><span><a href="#프로젝트-목표" data-toc-modified-id="프로젝트-목표-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>프로젝트 목표</a></span></li><li><span><a href="#데이터-읽기" data-toc-modified-id="데이터-읽기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>데이터 읽기</a></span><ul class="toc-item"><li><span><a href="#라이브러리-불러오기" data-toc-modified-id="라이브러리-불러오기-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>라이브러리 불러오기</a></span></li><li><span><a href="#데이터-불러오기" data-toc-modified-id="데이터-불러오기-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>데이터 불러오기</a></span></li></ul></li><li><span><a href="#정상성-확인" data-toc-modified-id="정상성-확인-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>정상성 확인</a></span><ul class="toc-item"><li><span><a href="#[TODO]--전체-데이터에서--MME017의-컬럼에-대해-정상성-확인해보기" data-toc-modified-id="[TODO]--전체-데이터에서--MME017의-컬럼에-대해-정상성-확인해보기-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>[TODO]  전체 데이터에서  MME017의 컬럼에 대해 정상성 확인해보기</a></span></li><li><span><a href="#전체-데이터-정상성-확인" data-toc-modified-id="전체-데이터-정상성-확인-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>전체 데이터 정상성 확인</a></span></li></ul></li><li><span><a href="#ARIMA-파라미터-찾기" data-toc-modified-id="ARIMA-파라미터-찾기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ARIMA 파라미터 찾기</a></span><ul class="toc-item"><li><span><a href="#TODO-최적의-파라미터-찾기" data-toc-modified-id="TODO-최적의-파라미터-찾기-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>[TODO] 최적의 파라미터 찾기</a></span></li></ul></li><li><span><a href="#이상치-찾기" data-toc-modified-id="이상치-찾기-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>이상치 찾기</a></span><ul class="toc-item"><li><span><a href="#최적의-파라미터로-ARIMA-모델-학습" data-toc-modified-id="최적의-파라미터로-ARIMA-모델-학습-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>최적의 파라미터로 ARIMA 모델 학습</a></span></li><li><span><a href="#임계값-구하기" data-toc-modified-id="임계값-구하기-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>임계값 구하기</a></span></li><li><span><a href="#이상치-표시하기" data-toc-modified-id="이상치-표시하기-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>이상치 표시하기</a></span></li></ul></li><li><span><a href="#전체-데이터-이상치-탐지-하기" data-toc-modified-id="전체-데이터-이상치-탐지-하기-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>전체 데이터 이상치 탐지 하기</a></span></li></ul></div>

## 프로젝트 개요
---

앞서 전처리를 끝내고 저장한 `MME_data.p`를 불러오고, 데이터의 정상성을 확인해봅니다.
또한, 정상성이 확인된 데이터에 대해 `ARIMA`로 예측해보고, 최적 성능을 내는 파라미터를 찾습니다.
그 후 학습된 ARIMA 모델로 이상치를 탐지해보도록 합니다.




## 데이터 읽기


### 라이브러리 설치

본 프로젝트를 시행하기 앞서서 라이브러리 설치 및 버전을 맞추어 줍니다. 

In [ ]:
%pip install pickle5

### 라이브러리 불러오기
---
프로젝트에 필요한 다양한 라이브러리, 파일을 불러옵니다.

In [ ]:
from statsmodels.tsa.stattools import adfuller # Augmented Dickey-Fuller test.
from statsmodels.tsa.arima.model import ARIMA

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle5 as pickle
import warnings
import itertools

warnings.filterwarnings(action='ignore') # warning ignore 

pd.options.display.max_rows = 80
pd.options.display.max_columns = 80
sns.set_style("whitegrid")

%matplotlib inline

### 데이터 불러오기
---

프로젝트 1에서 전처리가 끝난 데이터(`MME_data.p`) 를 불러옵니다. 또한 기기별로 이상치 탐지를 하기 위하여 기기 정보를 devices로 저장해 놓도록 합니다. 
 


In [ ]:
# load 
with open('MME_data.p', 'rb') as fp:
    total_df = pickle.load(fp)

In [ ]:
# 데이터를 쉽게 불러오기 위하여 기기를 정의
devices = [
    "MME017",
    "MME018",
    "MME071",
    "MME072",
    "MME073",
    "MME074",
    "MME091",
    "MME092",
    "MME093",
]

## 정상성 확인 

ARIMA 모델을 사용하기 위해선 `정상성`이 보장 되어야됩니다. 이를 위해 주어진 데이터가 정상성을 만족하는지 확인해봅니다. 

### [TODO]  전체 데이터에서  MME017의 컬럼에 대해 정상성 확인해보기  

- 전체 데이터는 total_df로 dictionary 형태입니다. 
- 해당 기기 이름은 MME017 입니다. 
- .plot()는 판다스의 그래프를 그려주는 내장함수입니다. 

In [ ]:
data = 'code'

result = 'code'

# 각각의 출력값 프린트
our_result = dict(
    zip(["adf", "pvalue", "usedlag", "nobs", "critical" "values", "icbest"], result)
)
print(our_result)

# 0.05 보다 작으면 정상성 만족
if our_result["pvalue"] < 0.05:
    print(f"본 컬럼은 정상성을 만족합니다.")
else:
    print(f"본 컬럼은 정상성을 만족하지 않습니다.")

### 전체 데이터 정상성 확인 

In [ ]:
def adfuller_test(data):
    """
        Summary: Augmented Dickey-Fuller test.를 이용한 정상성 확인
    Args: 
        기기별 데이터프레임
    return 
        정상성을 만족하지 못하는 컬럼 리스트
    """
    non_stationary_col = []

    # p-value가 0.05보다 작으면 null hypothesis를 기각.
    # null hypothesis 기각은 stationary 하다는 것
    reject_null = lambda pvalue: pvalue < 0.05

    # 모든 컬럼에 대해 조사하기
    for column in data:
        result = adfuller(data[column].values)  # Fuller test
        pvalue = result[1]

        if not reject_null(pvalue):
            print(f"{column:<30s} 컬럼은 정상성을 만족하지 않습니다.")
            non_stationary_col.append(column)

    return non_stationary_col

In [ ]:
for device in devices:
    print(f"[정상성 테스트] | 기기: {device}")
    data = total_df[device]
    adfuller_test(data)
    print()

## ARIMA 파라미터 찾기

전체 데이터에 대해 ACF 와 PACF를 확인하면서 파라미터를 찾기보다 가능한 모든 조합을 실행(Grid_Search)하도록합니다. 
모든 조합에 대해 시행 한 후 이에 따른 AIC 값이 제일 작은 파라미터를 최적의 파라미터로 간주하도록 합니다. 


### [TODO] 최적의 파라미터 찾기 

ARIMA모델에는 p,d,q 세개의 파라미터가 있습니다. 각각의 값의 따라 모델의 성능이 좌우 됩니다.

따라서 이번에는 각각의 모델을 학습시킬때 필요한 최적의 파라미터를 찾아보도록 하겠습니다.

- 전체 데이터 중 MME017 기기를 선택합니다. 
- 기기의 컬럼 중 `comb_atch_att_45005` 컬럼을 선택합니다. 이는 위에서 정상성 테스트를 통과한 컬럼 중 하나이기 때문입니다.
- 가능한 모든 조합에 대해 AIC 값을 출력합니다. (이때, p,d,q 는 0과 1 둘 중의 하나의 값을 갖습니다. 이에 따라 총 경우의 수는 $2^3$ 입니다.)
- 각각의 조합에 대해 ARIMA를 학습하고 파라미터와 AIC 값을 저장합니다. 
- 가장 작은 AIC값을 가지는 파라미터를 확인하고 `best_param`로 저장합니다 

In [ ]:
# 데이터 한 개 선택 
data  = 'code'

# 데이터 중 하나의 컬럼 선택 
data_col = 'code'

# 최적의 파라미터를 찾기 위한 Grid-Search 작업
p = range(0, 2)  # p - 며칠전 시점
d = range(0, 2)  # d - 차분
q = range(0, 2)  #

pdq = list(itertools.product(p, d, q))  # 모든 조합 만들기
for i in pdq:
    print(f"ARIMA pdq: {i}")

In [ ]:
# 결과값 저장 
params = []
results_AIC = []

# 모든 조합에 대해 최적의 파라미터 찾기 
for param in pdq: 
    # ARIMA 주어진 데이터로 학습 
    model = 'code'
    model_fit = model.fit()
    
    params.append(param) # 각각의 파라미터 값 저장 
    results_AIC.append('code') # AIC  값 저장 
            

In [ ]:
# 저장된 파라미터 값과 AIC 값 dataframe으로 표현
result_df = pd.DataFrame(
    [params, results_AIC], dtype="object", index=["params", "AIC"]
).T

In [ ]:
# 통계적 모델의 성능을 따질때에는 보통 AIC 가 낮은 값이 성능이 좋다고 볼수있다. 
result_df = result_df.sort_values(by='AIC', ascending=True)
result_df

In [ ]:
# 최적의 파라미터 추출
best_param = 'code'
best_param

## 이상치 찾기 

이번에는 학습 된 모델에서 `MSE(Mean Squared Error)`를 구하고, `MSE`가 일정 범위를 벗어난다면 이상치라고 판단하여 주어진 데이터에서 이상치를 표시해 보도록 하겠습니다. 

### 최적의 파라미터로 ARIMA 모델 학습 

앞서 찾은 최적의 파라미터`best_param`을 이용하여 ARIMA 모델을 학습합니다

In [ ]:
def ARIMA_forecast(data, best_param):
    """
    Summary:
        ARIMA 모델을 사용하여 데이터를 예측 
    Args:
        data
        best_param
    """
    #########################################
    # 1. Model Fitting
    #########################################
    model = ARIMA(
        data.values,
        order=best_param["params"],
        seasonal_order=(*best_param["params"], 12),
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    model_fit = model.fit()

    return model_fit

In [ ]:
model_fit = ARIMA_forecast(data_col, best_param)

### 임계값 구하기 

ARIMA 모델은 학습 후 `residual`(실제값과 예측값의 오차)를 기본 제공합니다. 이 값을 받아 제곱을하여 `MSE`값을 만듭니다. 
그 다음 `Threshold = MSE + standard deviation`로 설정을 하여 이 임계값을 넘는 값에 대해 이상치라고 판단합니다. 

In [ ]:
# 학습된 모델에서  residual 을 구합니다.
squared_errors = model_fit.resid ** 2

# residual의 평균 + z * residual의 편차 보다 크면 오차라고 판단합니다.
threshold = np.mean(squared_errors) + np.std(squared_errors)

### 이상치 표시하기 

임계값 보다 높은 데이터에 대해 이상치 데이터로 판단하고 그래프로 표시합니다

In [ ]:
# 임계치보다 큰 데이터에 대해 이상치로 판단합니다.   
data_indices = np.where(squared_errors >= threshold) # 인덱스를 저장 합니다. 

# 이상치를 넘는 인덱스를 가지고 abnorma_data_col을 만듭니다.  
abnorma_data_col = data_col.iloc[data_indices]

In [ ]:
# 그림 그리기
plt.plot(list(range(len(data_col.index.values))), data_col.values, color="green")
plt.scatter(data_indices[0], abnorma_data_col.values, color="r")
plt.show()

## 전체 데이터 이상치 탐지 하기 

위 과정을 모아 전체 데이터와 전체 컬럼에 대한 이상치 탐지를 합니다.

이를 위해 사용할 기기는 `MME017`로 고정하고, 사용할 컬럼은 정상성 테스트를 통과한 것들로 하겠습니다.

이후에 전체 기간에서 앞선 70%의 기간을 모델 학습용으로, 뒤의 30%는 테스트용으로 분리하겠습니다.

In [ ]:
device_df = total_df["MME017"]

non_stationary_col = adfuller_test(device_df)

# 정상성을 만족하지 않은 column들은 제외합니다.
device_df = device_df.drop(non_stationary_col, axis=1)

# 훈련 데이터를 70%, 테스트 데이터를 30%로 분리합니다.
train_size = int(len(device_df) * 0.7)

train_df = device_df.iloc[:train_size, :]
test_df = device_df.iloc[train_size:, :]

print(f"Train 데이터: {train_df.shape}, Test 데이터: {test_df.shape}")

In [ ]:
def find_best_param(data_col):
    best_param = None
    criteria = 1e9

    # 각각의 파라미터로 어떤 모델이 가장 성능이 좋은지 결과를 도출하는 것
    for param in pdq:
        model = ARIMA(
            data_col.values,
            order=param,
            seasonal_order=(*param, 12),
            enforce_stationarity=False,
            enforce_invertibility=False,
        )
        results = model.fit()

        # 통계적 모델의 성능을 따질때에는 보통 AIC 가 낮은 값이 성능이 좋다고 볼수있다.
        if criteria > results.aic:
            criteria = results.aic
            best_param = {"params": param}
            print(
                "| FIND BEST PARAMS | AIC: {:.2f} | p:{}, d:{}, q:{}".format(
                    criteria, *param
                ),
                end="\r",
            )

    return best_param

In [ ]:
def detect_anomalies(test_df, model_fit, z=0.7):
    pred = model_fit.forecast(steps=len(test_df))

    squared_errors = (test_df.values - pred) ** 2
    threshold = np.mean(squared_errors) + z * np.std(squared_errors)
    predictions = (squared_errors >= threshold).astype(int)

    results_df = test_df.reset_index()
    results_df.columns = ["ds", "y"]
    results_df["anomaly"] = predictions
    results_df["yhat_upper"] = results_df["y"]
    results_df["yhat_lower"] = results_df["y"]

    return results_df

In [ ]:
def dependency_plot(pred):
    total_indices = list(range(len(pred)))
    total_values = pred["y"].values

    abnormal_indices = pred[pred["anomaly"] != 0].index.values
    abnormal_values = pred[pred["anomaly"] != 0]["y"].values

    # 그림 그리기
    plt.plot(total_indices, total_values, color="green")
    plt.scatter(abnormal_indices, abnormal_values, color="r")
    plt.show()

In [ ]:
for idx, column in enumerate(device_df.columns, start=1):
    print(f"COLUMN {idx}: {column}")

    # ARIMA 학습
    best_param = find_best_param(train_df[column])  # 파라미터 찾기
    model_fit = ARIMA_forecast(train_df[column], best_param)  # ARIMA 모델 학습

    # ARIMA 예측 후 이상치 확인
    pred = detect_anomalies(test_df[column], model_fit)
    dependency_plot(pred)

print("--------------------------------------------------------")

---